In [ ]:
!pip install cryptography
%pylab inline
from tvb.interfaces.command.demos.importers.encrypt_data import *
import pyAesCrypt
from tvb.basic.profile import TvbProfile

# Encrypt data before uploading

TVB uses the RSA algorithm for asymmetric-key encryption and decryption of data. Unfortunately, RSA by itself is not enough because it can only encrypt limited amounts of data. 

To solve this problem, the data is encrypted using a symmetric-key encryption algorithm, namely AES, and the password which is used for generating the symmetric key is then encrypted using the public key of the RSA algorithm.

First, we read the public key.

In [ ]:
path_to_public_key = 'public_key.pem' # enter path to public key here

with open(path_to_public_key, "rb") as key_file:
    public_key = serialization.load_pem_public_key(
        key_file.read(),
        backend=default_backend()
    )

then we generate a random password.

In [ ]:
pass_size = TvbProfile.current.hpc.CRYPT_PASS_SIZE
password = generate_random_password(pass_size)

The next step is to encrypt the file(s) using the password generated at the previous step. Datatypes which require more files at uploading (e.g. CSV Connectivity) will need to be encrypted using the same password.

The encrypted content is saved at the location of the original data.

In [ ]:
paths_to_files = ['tvb_data/surfaceData/connectivity_76.zip'] # enter paths to data files here inside the list

buffer_size = TvbProfile.current.hpc.CRYPT_BUFFER_SIZE

for path in paths_to_files:
  encrypted_file_path = get_path_to_encrypt(path)
  pyAesCrypt.encryptFile(path, encrypted_file_path, password, buffer_size)

Asynchronously encrypt the password used at the previous step for the symmetric encryption

In [ ]:
password_bytes = str.encode(password)
encrypted_password = encrypt_password(public_key, password_bytes)

and save the encrypted password.

In [ ]:
encrypted_password_path = './' # enter path for saving the encrypted password
save_encrypted_password(encrypted_password, encrypted_password_path)
path_to_encrypted_password = os.path.join(encrypted_password_path, ENCRYPTED_PASSWORD_NAME)